In [1]:
API_KEY = 'a6af2051-a60d-4d5b-ae19-994ff9f22f24_up9205w'

In [3]:
import requests
import json

class Smartlead:
    BASE_URL = 'https://server.smartlead.ai/api/v1'

    def __init__(self, api_key):
        self.api_key = api_key

    # add create campaign method

    def get_campaign_sequences(self, campaign_id):
        url = f"{self.BASE_URL}/campaigns/{campaign_id}/sequences?api_key={self.api_key}"
        response = requests.get(url)
        return response.json()

    def get_campaign(self, campaign_id):
        url = f"{self.BASE_URL}/campaigns/{campaign_id}?api_key={self.api_key}"
        response = requests.get(url)
        return response.json()

    def get_campaign_email_accounts(self, campaign_id):
        url = f"{self.BASE_URL}/campaigns/{campaign_id}/email-accounts?api_key={self.api_key}"
        response = requests.get(url)
        return response.json()

    def get_campaign_leads(self, campaign_id, offset=0, limit=10):
        url = f"{self.BASE_URL}/campaigns/{campaign_id}/leads?api_key={self.api_key}&offset={offset}&limit={limit}"
        response = requests.get(url)
        return response.json()
    
    def post_campaign_leads(self, campaign_id, lead_list):
        """`lead_list` format is 
            
        lead_list = [
            {
                "first_name": "Cristiano",
                "last_name": "Ronaldo",
                "email": "cristiano@mufc.com",
                "phone_number": "0239392029",
                "company_name": "Manchester United",
                "website": "mufc.com",
                "location": "London",
                "custom_fields": {
                    "Title": "Regional Manager",
                    "First_Line": "Loved your recent post about remote work on Linkedin"
                },
                "linkedin_profile": "http://www.linkedin.com/in/cristianoronaldo",
                "company_url": "mufc.com"
            }
            # ... (add up to 100 leads)
        ]
        """
        settings = {
            "ignore_global_block_list": True,
            "ignore_unsubscribe_list": True,
            "ignore_duplicate_leads_in_other_campaign": False
        }
        if not isinstance(lead_list, list) or not isinstance(settings, dict):
            raise ValueError("lead_list must be a list and settings must be a dictionary.")

        if len(lead_list) > 100:
            raise ValueError("You can only send a maximum of 100 leads at a time.")

        url = f"{self.BASE_URL}/campaigns/{campaign_id}/leads?api_key={self.api_key}"
        data = {
            "lead_list": lead_list,
            "settings": settings
        }
        headers = {'Content-Type': 'application/json'}
        response = requests.post(url, headers=headers, data=json.dumps(data))
        return response.json()

# Example usage:
api_key = API_KEY
campaign_id = '123383'

smartlead = Smartlead(api_key)
sequences = smartlead.get_campaign_sequences(campaign_id)
# campaign = smartlead.get_campaign(campaign_id)
# email_accounts = smartlead.get_campaign_email_accounts(campaign_id)
# leads = smartlead.get_campaign_leads(campaign_id, offset=0, limit=10)

print(sequences)
# print(campaign)
# print(email_accounts)
# print(leads)


[{'id': 387060, 'created_at': '2023-10-27T18:11:13.050Z', 'updated_at': '2023-10-27T18:11:13.050Z', 'email_campaign_id': 123383, 'seq_number': 1, 'seq_delay_details': {'delayInDays': 1}, 'subject': '{{subject_line}}', 'email_body': '<div>{{body}}&nbsp;</div>', 'sequence_variants': None}]


In [5]:
lead_list = [
    {
        "first_name": "Cristiano",
        "last_name": "Ronaldo",
        "email": "cristiano@mufc.com",
        "phone_number": "0239392029",
        "company_name": "Manchester United",
        "website": "mufc.com",
        "location": "London",
        "custom_fields": {
            "Title": "Regional Manager",
            "First_Line": "Loved your recent post about remote work on Linkedin"
        },
        "linkedin_profile": "http://www.linkedin.com/in/cristianoronaldo",
        "company_url": "mufc.com"
    }
    # ... (add up to 100 leads)
]



response = smartlead.post_campaign_leads(campaign_id, lead_list)


In [8]:
from bs4 import BeautifulSoup


def get_lead_data(lead_address):
    LEAD_URL = f"https://server.smartlead.ai/api/v1/leads/?api_key={API_KEY}&email={lead_address}"

    response = requests.get(LEAD_URL)
    response = response.json()
    return response

def get_convo_history_using_lead_id(lead_id: int, campaign_id: int):
    CONVO_URL = f"https://server.smartlead.ai/api/v1/campaigns/{campaign_id}/leads/{lead_id}/message-history?api_key={API_KEY}"
    
    response = requests.get(CONVO_URL)
    response = response.json()
    return response

def reply_to_convo(
        campaign_id: int,
        email_stats_id: str,
        email_body: str,
        reply_message_id: str,
        reply_email_time: str,
        reply_email_body: str,
    ):
    REPLY_URL = f"https://server.smartlead.ai/api/v1/campaigns/{campaign_id}/reply-email-thread?api_key={API_KEY}"
    data = {
        "email_stats_id": email_stats_id,
        "email_body": email_body,
        "reply_message_id": reply_message_id,
        "reply_email_time": reply_email_time,
        "reply_email_body": reply_email_body
    }
    return data
    headers = {'Content-Type': 'application/json'}
    response = requests.post(REPLY_URL, headers=headers, data=json.dumps(data))
    return response.json()

lead_data = get_lead_data('david@sellscale.com')
lead_id = lead_data['id']

convo_history = get_convo_history_using_lead_id(lead_id, 156597)
history = convo_history.get('history')

# Work backwards from history
for item in history[::-1]:
    if item['type'] == 'REPLY':
        stats_id = item['email_stats_id']
        reply_message_id = item['message_id']
        reply_email_time = item['time']
        reply_email_body = item['email_body']

        break

# for item in history:
#     print(item)
#     bs = BeautifulSoup(item["email_body"], 'html.parser')
#     remove_past_convo = bs.find('div', {'class': 'gmail_quote'})
#     if remove_past_convo:
#         remove_past_convo.decompose()
#     print(bs.get_text()[:150])




{'stats_id': '478ecf04-8590-48e7-aaf1-270d1555dcf2', 'type': 'SENT', 'message_id': '<sw-478ecf04-8590-48e7-aaf1-270d1555dcf2@getdoppler.net>', 'time': '2023-12-05T18:24:16.928Z', 'email_body': '<div fr-original-style="" style="-webkit-tap-highlight-color: transparent; box-sizing: inherit;">This is a test, hope you\'re doing well David :).</div>', 'subject': 'Hi David'}
{'stats_id': '478ecf04-8590-48e7-aaf1-270d1555dcf2', 'type': 'REPLY', 'message_id': '<3ec172b7-83a5-4303-af89-743105fc242a@Spark>', 'time': '2023-12-05T21:29:11.000Z', 'email_body': '<html xmlns="http://www.w3.org/1999/xhtml">\r\n<head>\r\n<title></title>\r\n</head>\r\n<body>\r\n<div name="messageBodySection">\r\n<div dir="auto">Hey there, I\'m doing great! What about yourself?</div>\r\n</div>\r\n<div name="messageSignatureSection"><br />\r\n<font color="#10B981"><b><span style="font-size: 16px;"><span style="font-family: Helvetica , Arial , sans-serif;">David Wei&#160;</span></span></b></font><span style="font-family: H

In [21]:
from typing import Optional

from bs4 import BeautifulSoup


class SmartleadCampaignStatisticEntry:
    """This class represents a data entry from the Smartlead Campaign Statistics API."""

    def __init__(
        self,
        lead_name: Optional[str] = None,
        lead_email: Optional[str] = None,
        lead_category: Optional[str] = None,
        sequence_number: Optional[int] = None,
        stats_id: Optional[int] = None,
        email_campaign_seq_id: Optional[int] = None,
        seq_variant_id: Optional[int] = None,
        email_subject: Optional[str] = None,
        email_message: Optional[str] = None,
        sent_time: Optional[str] = None,
        open_time: Optional[str] = None,
        click_time: Optional[str] = None,
        reply_time: Optional[str] = None,
        open_count: Optional[int] = None,
        click_count: Optional[int] = None,
        is_unsubscribed: Optional[bool] = None,
        is_bounced: Optional[bool] = None,
    ):
        self.lead_name = lead_name
        self.lead_email = lead_email
        self.lead_category = lead_category
        self.sequence_number = sequence_number
        self.stats_id = stats_id
        self.email_campaign_seq_id = email_campaign_seq_id
        self.seq_variant_id = seq_variant_id
        self.email_subject = email_subject
        self.email_message = email_message
        self.sent_time = sent_time
        self.open_time = open_time
        self.click_time = click_time
        self.reply_time = reply_time
        self.open_count = open_count
        self.click_count = click_count
        self.is_unsubscribed = is_unsubscribed
        self.is_bounced = is_bounced

    def to_dict(self):
        return {
            "lead_name": self.lead_name,
            "lead_email": self.lead_email,
            "lead_category": self.lead_category,
            "sequence_number": self.sequence_number,
            "stats_id": self.stats_id,
            "email_campaign_seq_id": self.email_campaign_seq_id,
            "seq_variant_id": self.seq_variant_id,
            "email_subject": self.email_subject,
            "email_message": self.email_message,
            "sent_time": self.sent_time,
            "open_time": self.open_time,
            "click_time": self.click_time,
            "reply_time": self.reply_time,
            "open_count": self.open_count,
            "click_count": self.click_count,
            "is_unsubscribed": self.is_unsubscribed,
            "is_bounced": self.is_bounced,
        }
    

CAMPAIGN_STATS_URL = f"https://server.smartlead.ai/api/v1/campaigns/132424/statistics?api_key={API_KEY}"

response = requests.get(CAMPAIGN_STATS_URL)
response = response.json()

data = response['data']

sent_count = 0
open_count = 0
reply_count = 0
for lead in data:
    lead: SmartleadCampaignStatisticEntry = SmartleadCampaignStatisticEntry(**lead)
    # print(lead.lead_email)
    if lead.lead_email == 'elamoureux@kirby.org':
        print(lead.email_subject)

        message_newlined = lead.email_message.replace('<br>', '\n')
        bs = BeautifulSoup(message_newlined, 'html.parser')
        print(bs.get_text())

    if lead.sent_time:
        sent_count += 1

    if lead.open_time:
        open_count += 1

    if lead.reply_time:
        reply_count += 1

# print(f"Sent: {sent_count}")
# print(f"Open: {open_count}")
# print(f"Reply: {reply_count}")

fellow ucsc alum - small ask
Hi Emily,
I’m a fellow Slug reaching out with a small ask - would you be willing to loop me in with whoever oversees application security and config at Kirby?
My company helps developers and security teams synchronize their secrets and app configuration across various devices, environments, and team members. We're bringing a new approach to handling .env files, helping with efficiency and security.
I'd love the opportunity to assist a fellow alum and everyone at Kirby!
- Adam
​follow-up: ucsc alum
Hi Emily ,
Wanted to do a friendly follow-up - any chance you could point me towards who oversees application security / config at Kirby? Just wanted to do a friendly follow-up.
- Adam
PS - if I missed the mark, simply reply 'unsub' and I'll stop emailing.


In [12]:
def add_leads_to_campaign_by_id(campaign_id: int, lead_list: list):
    ADD_LEADS_TO_CAMPAIGN_BY_ID_URL = (
        f"https://server.smartlead.ai/api/v1/campaigns/{campaign_id}/leads?api_key={API_KEY}"
    )

    data = {
        "lead_list": lead_list,
        "settings": {
            "ignore_global_block_list": False,
            "ignore_unsubscribe_list": False,
            "ignore_duplicate_leads_in_other_campaign": True,
        },
    }

    headers = {"Content-Type": "application/json"}
    response = requests.post(
        ADD_LEADS_TO_CAMPAIGN_BY_ID_URL,
        headers=headers,
        data=json.dumps(data),
    )
    return response.json()

lead_list = [
    {
        "email": "ishan@sellscale.com",
        "custom_fields": {
            "Subject_Line": "We are running an import test!",
            "Body_1": "This email was sent by Smartlead after we had automatically imported a list of leads into the campaign. Ooo look, some <b>bold text</b>!",
            "Body_2": "This is the second body paragraph. We can also add <a href='https://www.google.com'>links</a>!",
        }
    }

]
add_leads_to_campaign_by_id(163705, lead_list)

{'ok': True,
 'upload_count': 1,
 'total_leads': 1,
 'block_count': 0,
 'duplicate_count': 0,
 'invalid_email_count': 0,
 'invalid_emails': [],
 'already_added_to_campaign': 0,
 'unsubscribed_leads': [],
 'is_lead_limit_exhausted': False,
 'lead_import_stopped_count': 0}

In [22]:
import requests
from datetime import datetime

API_KEY = "a6af2051-a60d-4d5b-ae19-994ff9f22f24_up9205w"


def create_campaign(name: str):
    CREATE_CAMPAIGN_URL = (
        f"https://server.smartlead.ai/api/v1/campaigns/create?api_key={API_KEY}"
    )

    data = {
        "name": name,
    }

    headers = {"Content-Type": "application/json"}
    response = requests.post(
        CREATE_CAMPAIGN_URL,
        headers=headers,
        data=json.dumps(data),
    )
    return response.json()


def fetch_campaign_sequence(campaign_id: int):
    FETCH_CAMPAIGN_SEQUENCE_URL = f"https://server.smartlead.ai/api/v1/campaigns/{campaign_id}/sequences?api_key={API_KEY}"

    response = requests.get(FETCH_CAMPAIGN_SEQUENCE_URL)
    return response.json()


def save_campaign_sequence(campaign_id: int, sequences: list):
    SAVE_CAMPAIGN_SEQUENCE_URL = f"https://server.smartlead.ai/api/v1/campaigns/{campaign_id}/sequences?api_key={API_KEY}"

    data = {
        "sequences": sequences,
    }

    headers = {"Content-Type": "application/json"}
    response = requests.post(
        SAVE_CAMPAIGN_SEQUENCE_URL,
        headers=headers,
        data=json.dumps(data),
    )
    return response.json()


def get_email_accounts():
    GET_EMAIL_ACCOUNTS_URL = (
        f"https://server.smartlead.ai/api/v1/email-accounts/?api_key={API_KEY}"
    )

    response = requests.get(GET_EMAIL_ACCOUNTS_URL)
    result = response.json()

    aakash_emails = []
    for account in result:
        if account["from_name"] == "Aakash Adesara":
            aakash_emails.append(account)
    return aakash_emails


def add_email_account_to_campaign(campaign_id: int, email_account_ids: int):
    ADD_EMAIL_ACCOUNT_TO_CAMPAIGN_URL = f"https://server.smartlead.ai/api/v1/campaigns/{campaign_id}/email-accounts?api_key={API_KEY}"

    data = {
        "email_account_ids": email_account_ids,
    }

    headers = {"Content-Type": "application/json"}

    response = requests.post(
        ADD_EMAIL_ACCOUNT_TO_CAMPAIGN_URL,
        headers=headers,
        data=json.dumps(data),
    )
    return response.json()


def update_campaign_schedule(campaign_id: int, schedule: dict):
    UPDATE_CAMPAIGN_SCHEDULE_URL = f"https://server.smartlead.ai/api/v1/campaigns/{campaign_id}/schedule?api_key={API_KEY}"

    data = schedule

    headers = {"Content-Type": "application/json"}

    response = requests.post(
        UPDATE_CAMPAIGN_SCHEDULE_URL,
        headers=headers,
        data=json.dumps(data),
    )
    return response.json()

def patch_campaign_status(campaign_id: int, status: str):
    PATCH_CAMPAIGN_STATUS_URL = f"https://server.smartlead.ai/api/v1/campaigns/{campaign_id}/status?api_key={API_KEY}"

    data = {
        "status": status,
    }

    headers = {"Content-Type": "application/json"}

    response = requests.patch(
        PATCH_CAMPAIGN_STATUS_URL,
        headers=headers,
        data=json.dumps(data),
    )
    return response.json()


# create_campaign("test create campaign")
id = 163705
sequences = [
    {
        "seq_delay_details": {"delay_in_days": 20},
        "seq_number": 1,
        "subject": "{{Subject_Line}}",
        "email_body": "{{Body_1}}",
    },
    {
        "seq_delay_details": {"delay_in_days": 20},
        "seq_number": 2,
        "subject": "",
        "email_body": "{{Body_2}}",
    },
]
# save_campaign_sequence(
#     id, sequences
# )

# aakash_emails = get_email_accounts()
# email_ids = [email["id"] for email in aakash_emails]
# add_email_account_to_campaign(id, email_ids)
# campaign_schedule = {
#     "timezone": "America/Los_Angeles",
#     "days_of_the_week": [1, 2, 3],  # [0,1,2,3,4,5,6]
#     "start_hour": "09:00",  # "09:00"
#     "end_hour": "18:00",  # "18:00"
#     "min_time_btw_emails": 10,  # time in minutes between emails
#     "max_new_leads_per_day": 20,  # max new leads per day
#     "schedule_start_time": datetime.now().isoformat(),  # Standard ISO format accepted
# }
# update_campaign_schedule(id, campaign_schedule)

patch_campaign_status(id, "START")

{'ok': True}